# Access non-IPv6 services (i.e. GitHub) from IPv6 FABRIC nodes.

Based on [https://nat64.net/](https://nat64.net/)

Note that NAT64 could go away at any minute, as it is a public service ran independently by Kasper Dupont, and is not affiliated with FABRIC.

## Import the FABlib Library

The `show_config` method shows what some environment variables relevant to your FABRIC user are set to. All necessary variables may be set [here](../../../fabric_examples/fablib_api/configure_environment/configure_environment.ipynb).

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

## Create a Node

In [ ]:
try:
    #Create a slice
    slice = fablib.new_slice(name="MySlice")

    # Add a node. (We want an IPv6 site so avoid IPv4 sites)
    # Add a node. TACC and MAX are both IPv4 sites. Since we
    # want an IPv6 site, we will avoid those two sites.
    slice.add_node(name="Node1", avoid=['TACC','MAX'])

    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

### Print the node


In [ ]:
try:
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Upload and Execute the NAT64 Script

We will use a NAT64 script, which configures an *IPv6* node so that it can access non-*IPv6* services. To view the script, click [here](./nat64.sh). This script sets up the [Public NAT64 Service](https://nat64.net) on your node.

For more details on how uploading and executing scripts works on FABRIC, click [here](../upload_and_execute/upload_and_execute.ipynb).

In [ ]:
from ipaddress import ip_address, IPv6Address

try:
    node = slice.get_node(name="Node1")     
    
    # If the node is an IPv6 Node then configure NAT64
    if type(ip_address(node.get_management_ip())) is IPv6Address:
        node.upload_file('nat64.sh', 'nat64.sh')
        
        stdout, stderr = node.execute(f'chmod +x nat64.sh && ./nat64.sh')
        print(stdout)
        print(stderr)
        
    # Access non-IPv6 Services
    stdout, stderr = node.execute(f'sudo yum install -y -q git && git clone https://github.com/fabric-testbed/jupyter-examples.git')
    print(stdout)
    print(stderr)
    
    stdout, stderr = node.execute(f'ls jupyter-examples')
    print(stdout)
    print(stderr)
     
        
except Exception as e:
    print(f"Exception: {e}")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")